In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image



device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 20
batch_size = 4
epochs = 2000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        #print(data_number)
        xs_0=list()
        for i in range(20): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            
            r=open ('data/'+str(i+20*j)+'_building_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float32)
            temp_greed=np.array(temp_green, dtype=np.float32)
            temp_blue=np.array(temp_blue, dtype=np.float32)

            xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<20:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                else:
                    if i==1:
                        ys_0.append([float(E_r)])

                    else:

                        ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train = np.array(xs_0_, dtype=np.float32)
y_train = np.array(ys_0, dtype=np.float32)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()

xs_0=list()
xs_0_=list()
xs_1=list()
"""
for j in range(data_number): 
    xs_0=list()
    for i in range(10): 
        temp_red=list()
        temp_blue=list()
        temp_green=list()

        r=open ('data/'+str(i+10*j)+'_r.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_red.append(target)

        r=open ('data/'+str(i+10*j)+'_g.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_green.append(target)

        r=open ('data/'+str(i+10*j)+'_b.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_blue.append(target)

        temp_red=np.array(temp_red, dtype=np.float32)
        temp_greed=np.array(temp_green, dtype=np.float32)
        temp_blue=np.array(temp_blue, dtype=np.float32)

        xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
        xs_1=list(xs_1) 
        xs_0.append(xs_1)
    
    xs_0_.append(xs_0)
        
x_test = np.array(xs_0_, dtype=np.float32)
x_test = torch.from_numpy(x_test).cuda()
"""
#print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#dataset_test = TensorDataset(x_test, y_train)
#dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
#print(x_test.shape)

xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

data_ready


In [10]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,5,5,stride=1)
        self.layer_2 = nn.Conv2d(5,2,5,stride=2)
        #self.layer_3 = nn.Conv2d(13,20,7,stride=3)
        #self.layer_4 = nn.Conv2d(20,30,5,stride=2)
        #self.layer_5 = nn.Conv2d(30,40,5,stride=2)
        self.layer_6 = nn.LSTM(8832,100,1, batch_first = True)
        self.layer_7 = nn.Linear(100, 1)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        
        self.a=0
        self.b=0
        self.c=0
        #self.d=0
        #self.e=0
        self.count_=0
        self.o=0
        
    def forward(self, x):
        x = x.reshape((-1, 3, 100, 200))
        x = self.layer_1(x)
        self.a = self.relu(x)
        x = self.layer_2(self.a)
        self.b = self.relu(x)
        #x = self.layer_3(self.b)
        #self.c = self.relu(x)
        #x = self.layer_4(self.c)
        #self.d = self.relu(x)
        #x = self.layer_5(self.d)
        #self.e = self.relu(x)
        x = self.b.reshape((self.batch_size, self.length, x.shape[-3],x.shape[-2],x.shape[-1]))
        x = x.reshape((self.batch_size, self.length, x.shape[-3]*x.shape[-2]*x.shape[-1]))
        self.o, (h_n, c_n) = self.layer_6(x)
        #print(x.shape)
        
        #x = self.relu(h_n[-1,:,:].reshape((self.batch_size,-1)))
        x = self.layer_7(self.o[:,1:,:]).reshape((self.batch_size,-1))
        #print(x.shape)
        #x.retain_grad()
        self.count_+=1
        #print("종우",x.grad)
        return x

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
#print("서",model.parameters().shape)
      
def train(dataloader, model, loss, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        
        #print(pred.shape,y.shape)
        loss_result = loss(pred, y)
        optimizer.zero_grad()
        
        model.a.retain_grad()
        model.b.retain_grad()
        #model.c.retain_grad()
        #model.d.retain_grad()
        #model.e.retain_grad()
        model.o.retain_grad()
        #print(model.e.grad)
        loss_result.backward()
        #print(model.e.requires_grad)
        #print(model.e.grad)
        #print(model.b.grad)
        #print(model.c.grad)
        #print(model.d.grad)
        #print(model.e.grad)
        #print(loss_result.sum())
        
        
        
        #print("종우",loss_result.register_hook())
    
    #print(result_temp.shape, result_temp)
    #print(result_temp)
    #print(param.shape)
    #print(result_temp.shape)
    #print(model.)
    #print(model.e.grad.shape, result_temp.shape)
    if model.count_==5:
        temp_sum=0
        #print(model.a.grad.shape)
        width_ = model.a.grad.shape[-1]
        #print(width_)
        result_temp = torch.sum(model.a.grad,-1)
        #print(result_temp)
        height_ = result_temp.shape[-1]
        #print(height_)
        result_temp = torch.sum(result_temp,-1)
        #print(result_temp.shape, result_temp)
        #result_temp_=0
        #for i in range(model.batch_size):
        #    result_temp_ += result_temp[15+i]
        result_temp_ = torch.sum(result_temp,0)
        result_temp = result_temp_/5
        #print(result_temp.shape,model.a.grad.shape)
        #print(result_temp.shape, result_temp)
        model.count_=0
        print(result_temp)
        temp_sum=0
        for k in range(model.batch_size):  
        #for j in range(1):
            #print(result_temp.shape[1])
            #print(result_temp.shape[1])
            #j=10
            #for k in range(1): 
            #temp_sum=0
            #print(model.a.grad.shape)
            for j in range(result_temp.shape[0]):
                k=10
                #print(result_temp[k,j].shape, model.a.grad.shape)
                #print(model.a.grad[j].shape)
                #print(i,j,k)
                #print(result_temp[k,j].shape,model.e.grad[k,j,:,:].shape)
                temp_sum += result_temp[j]*(model.a.grad[15+k,j,:,:])
            #print(result_temp.shape)
        result_ = model.relu(temp_sum)
        result_ = result_.cpu().numpy()
        min_= np.min(result_)
        max_ = np.max(result_)
        #print(model.o.grad)
        #print(model.b.grad)
        #print(model.a.grad)
        #print(min_,max_,np.sum(result_))
        #print(result_)
        result_ = ((255*result_)/max_)  #.astype(np.uint8)
        min_= np.min(result_)
        max_ = np.max(result_)
        #print(model.o.grad)
        #print(model.b.grad)
        #print(model.a.grad)
        #print(min_,max_,result_)
        pil_image=Image.fromarray(result_)
        pil_image.show()
        with open("data/"+str(i)+"_r.csv",'a',newline='') as aa:
            writer_a = csv.writer(aa, delimiter=',')

            #print(temp_sum)
    #print(temp_sum.shape, temp_sum)
    #result_ = model.relu(temp_sum)
    #print(result_.shape)
    #temp_sum= self.relu(temp_sum)
    #result_ = result_.cpu().numpy()
    #min_= np.min(result_)
    #max_ = np.max(result_)
    #print(min_,max_)
    #pil_image=Image.fromarray(result_)
    #pil_image.show()
    #print(result_)
    #print(model.e.grad)
    
        
    optimizer.step()
            
def test(dataloader, model, loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss, optimizer)
    #test(dataloader_test, model, loss)
print("Done!")


Epoch 1
-------------------------------
tensor([-0.0226,  0.0257,  0.0002,  0.0287, -0.0086], device='cuda:0')
Epoch 2
-------------------------------
tensor([-0.0197,  0.0058,  0.0136,  0.0231, -0.0162], device='cuda:0')
Epoch 3
-------------------------------
tensor([-0.0157, -0.0003,  0.0145,  0.0172, -0.0152], device='cuda:0')
Epoch 4
-------------------------------
tensor([-0.0227,  0.0036,  0.0172,  0.0241, -0.0193], device='cuda:0')
Epoch 5
-------------------------------
tensor([-0.0110,  0.0091,  0.0019,  0.0114, -0.0049], device='cuda:0')
Epoch 6
-------------------------------
tensor([ 0.0010,  0.0039, -0.0042, -0.0008,  0.0031], device='cuda:0')
Epoch 7
-------------------------------
tensor([ 0.0142,  0.0050, -0.0162, -0.0136,  0.0155], device='cuda:0')
Epoch 8
-------------------------------
tensor([-0.0031,  0.0028,  0.0003,  0.0032, -0.0012], device='cuda:0')
Epoch 9
-------------------------------
tensor([ 0.0088,  0.0185, -0.0237, -0.0090,  0.0190], device='cuda:0')
E

KeyboardInterrupt: 

In [ ]:
import torch
import cv2
import numpy as np
from torchsummary import summary
from torch.nn import functional as F

class ModelOutputs_resnet():
    def __init__(self, model, target_layers, target_sub_layers):
        self.model = model
        self.target_layers = target_layers
        self.target_sub_layers = target_sub_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def get_gradients(self):
        return self.gradients

    def __call__(self, x):
        self.gradients = []
        for name, module in self.model.named_children(): # 모든 layer에 대해서 직접 접근
            x = module(x)
            if name== 'avgpool': # avgpool이후 fully connect하기 전 data shape을 flatten시킴
                x = torch.flatten(x,1)
            if name in self.target_layers: # target_layer라면 해당 layer에서의 gradient를 저장
                for sub_name, sub_module in module[len(module)-1].named_children():
                    if sub_name in self.target_sub_layers:
                        x.register_hook(self.save_gradient) #
                        target_feature_maps = x # x's shape = 512X14X14(C,W,H) feature map
        return target_feature_maps, x # target_activation : target_activation_layer's feature maps // output : classification ( ImageNet's classes : 1000 )


class GradCam_resnet:
    def __init__(self, model, target_layer_names,target_sub_layer_names, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:  # GPU일 경우 model을 cuda로 설정
            self.model = model.cuda()

        self.extractor = ModelOutputs_resnet(self.model, target_layer_names,target_sub_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):

        if self.cuda:  # GPU일 경우 input을 cuda로 변환하여 전달
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)

        probs,idx = 0, 0
        if index == None:
            index = np.argmax(output.cpu().data.numpy())  # index = 정답이라고 추측한 class index
            h_x = F.softmax(output,dim=1).data.squeeze()
            probs, idx = h_x.sort(0,True)


        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1  # 정답이라고 생각하는 class의 index 리스트 위치의 값만 1로
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)  # numpy배열을 tensor로 변환
        # requires_grad == True 텐서의 모든 연산에 대하여 추적
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features  # A^k

        target_cam = target.cpu().data.numpy()
        bz, nc, h,w = target_cam.shape

        target = target.cpu().data.numpy()[0, :]

        params = list(self.model.parameters())

        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())

        cam = weight_softmax[index].dot(target_cam.reshape((nc,h*w)))
        cam = cam.reshape(h,w)
        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, (224, 224))  # 224X224크기로 변환
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)


        weights = np.mean(grads_val, axis=(2, 3))[0, :]  # 논문에서의 global average pooling 식에 해당하는 부분
        grad_cam = np.zeros(target.shape[1:], dtype=np.float32)  # 14X14

        for i, w in enumerate(weights): # calcul grad_cam
            grad_cam += w * target[i, :, :]  # linear combination L^c_{Grad-CAM}에 해당하는 식에서 ReLU를 제외한 식

        grad_cam = np.maximum(grad_cam, 0)  # 0보다 작은 값을 제거
        grad_cam = cv2.resize(grad_cam, (224, 224))  # 224X224크기로 변환
        grad_cam = grad_cam - np.min(grad_cam)  #
        grad_cam = grad_cam / np.max(grad_cam)  # 위의 것과 해당 줄의 것은 0~1사이의 값으로 정규화하기 위한 정리
        return grad_cam, cam ,index,probs,idx ,

In [ ]:
import torch
import cv2
import numpy as np



from torch.nn import functional as F


class FeatureExtractor_vgg():
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers): # target_layers = 35 ==> VGG19에서 가장 마지막 MaxPool2D전 ReLU함수
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)


    def __call__(self, x):
        self.gradients = []
        for name, module in self.model._modules.items(): # 모든 layer에 대해서 직접 접근
            x = module(x)
            if name in self.target_layers: # target_layer라면 해당 layer에서의 gradient를 저장
                x.register_hook(self.save_gradient) #
                target_feature_maps = x # x's shape = 512X14X14(C,W,H) feature map
        return target_feature_maps, x


class ModelOutputs_vgg():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, target_layers):
        self.model = model
        self.feature_extractor = FeatureExtractor_vgg(self.model.features, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations, output = self.feature_extractor(x)
        output = output.view(output.size(0), -1)
        output = self.model.classifier(output) # feature extract를 통해서 나온 값을 활용하여 classification 진행
        #print("ModelOutputs().output.shape : ",output[0])
        #print("ModelOutputs().target_activations.shape :",target_activations[0])
        return target_activations, output

class GradCam_vgg:
    def __init__(self, model, target_layer_names, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda: # GPU일 경우 model을 cuda로 설정
            self.model = model.cuda()

        self.extractor = ModelOutputs_vgg(self.model, target_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda: # GPU일 경우 input을 cuda로 변환하여 전달
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)
        #print("features : ",features.cpu().data.numpy().shape) # 해당 위치에서 추출된 feature map ( 512,14,14 ) (ChannelX14X14)
        #print("output : ",output.cpu().data.numpy().shape) # class를 의미함
        probs, idx = 0,0
        #print("index : ", index)
        if index == None:
            index = np.argmax(output.cpu().data.numpy())  # index = 정답이라고 추측한 class index
            h_x = F.softmax(output,dim=1).data.squeeze()
            probs, idx = h_x.sort(0,True)
        #print("index : ", index)
        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1 # 정답이라고 생각하는 class의 index 리스트 위치의 값만 1로
        one_hot = torch.from_numpy(one_hot).requires_grad_(True) # numpy배열을 tensor로 변환
        # requires_grad == True 텐서의 모든 연산에 대하여 추적
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        self.model.features.zero_grad()
        self.model.classifier.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()
        #print("grads_val : ",grads_val.shape) # 512 X 14 X 14
        target = features  # A^k
        target = target.cpu().data.numpy()[0, :]

        cam = None

        weights = np.mean(grads_val, axis=(2, 3))[0, :]  # 논문에서의 global average pooling 식에 해당하는 부분
        grad_cam = np.zeros(target.shape[1:], dtype=np.float32)  # 14X14

        for i, w in enumerate(weights):  # calcul grad_cam
            grad_cam += w * target[i, :, :]  # linear combination L^c_{Grad-CAM}에 해당하는 식에서 ReLU를 제외한 식

        grad_cam = np.maximum(grad_cam, 0)  # 0보다 작은 값을 제거
        grad_cam = cv2.resize(grad_cam, (224, 224))  # 224X224크기로 변환
        grad_cam = grad_cam - np.min(grad_cam)  #
        grad_cam = grad_cam / np.max(grad_cam)  # 위의 것과 해당 줄의 것은 0~1사이의 값으로 정규화하기 위한 정리
        return grad_cam, cam, index, probs, idx,